In [24]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [25]:
mnist = input_data.read_data_sets('MNIST_data',one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [26]:
#60000张训练集，10000张测试集合，5000验证集合
#每个批次大小
batch_size = 100
#计算一下一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

In [27]:
print(mnist.train.num_examples)
print(mnist.test.num_examples)
print(mnist.validation.num_examples)

55000
10000
5000


In [38]:
#定义两个placeholder 
#把批次的图片传进去，开始是100张一次的传送，直到最后
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
#定义一个dropout的值
keep_prob = tf.placeholder(tf.float32)
#第一层
W1 = tf.Variable(tf.truncated_normal([784,1000],stddev=0.1))#改成截断正太分布
b1 = tf.Variable(tf.zeros([1000])+0.1)
output1 = tf.matmul(x,W1) + b1
L1 = tf.nn.tanh(output1)
#L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
L1_drop = tf.nn.dropout(L1,keep_prob)

#第二层
W2 = tf.Variable(tf.truncated_normal([1000,2000],stddev=0.1))
b2 = tf.Variable(tf.zeros([2000])+0.2)
output2 = tf.matmul(L1_drop,W2)+b2
L2 = tf.nn.tanh(output2)
#L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
L2_drop = tf.nn.dropout(L2,keep_prob)









#第三层
W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b3 = tf.Variable(tf.zeros([1000])+0.1)
output3 = tf.matmul(L2_drop,W3)+b3
L3 = tf.nn.tanh(output3)
#L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
L3_drop = tf.nn.dropout(L3,keep_prob)

# 第四层
W4 = tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b4 = tf.Variable(tf.zeros([10])+0.1)
output4 = tf.matmul(L3_drop,W4)+ b4















#预测分类
prediction = tf.nn.softmax(output4)

#定义一个损失函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits = prediction))#联合交叉熵
#loss = tf.reduce_mean(tf.square(y-prediction))
#定义一个方法来收敛神经网络，使用梯度下降法,学习率=0.1
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
#train_step = tf.train.AdamOptimizer(0.1).minimize(loss)
#初始化
init = tf.global_variables_initializer()

#定义一个求准确率的方法
#如果有两个相同的就会返回TRUE，不相同就返回false，然后就依次存入correct_prediction
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中的最大值

#求准确率
#首先把bool值转化成32位的浮点数，然后求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [39]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):#把所有的图片训练20次
        #一批一批的迭代图片，一次迭代100张，一共运行n_batch次
        for batch in range(n_batch):
            #每次传入一百张图片
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:0.7})
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:0.7})
        print('迭代次数=' + str(epoch) + '测试准确率=' + str(acc)+'训练准确率='+ str(train_acc))

迭代次数=0测试准确率=0.8605训练准确率=0.8532364
迭代次数=1测试准确率=0.8911训练准确率=0.8849091
迭代次数=2测试准确率=0.8979训练准确率=0.8970909
迭代次数=3测试准确率=0.9057训练准确率=0.9062
迭代次数=4测试准确率=0.9119训练准确率=0.9105273
迭代次数=5测试准确率=0.9136训练准确率=0.91505456
迭代次数=6测试准确率=0.9185训练准确率=0.9181091
迭代次数=7测试准确率=0.9222训练准确率=0.92252725
迭代次数=8测试准确率=0.9237训练准确率=0.9272364
迭代次数=9测试准确率=0.9236训练准确率=0.9280546


In [ ]:
#没有dropout
# 迭代次数=0测试准确率=0.9206训练准确率=0.93176365
# 迭代次数=1测试准确率=0.9374训练准确率=0.9562727
# 迭代次数=2测试准确率=0.9439训练准确率=0.9672727
# 迭代次数=3测试准确率=0.949训练准确率=0.97418183
# 迭代次数=4测试准确率=0.9522训练准确率=0.97894543
# 迭代次数=5测试准确率=0.954训练准确率=0.98218185
# 迭代次数=6测试准确率=0.9564训练准确率=0.98441815
# 迭代次数=7测试准确率=0.9579训练准确率=0.9864
# 迭代次数=8测试准确率=0.9596训练准确率=0.9878182
# 迭代次数=9测试准确率=0.9604训练准确率=0.98896366
# 迭代次数=10测试准确率=0.9607训练准确率=0.9899455

In [ ]:
#dropout 30%
# 迭代次数=0测试准确率=0.7816训练准确率=0.7733455
# 迭代次数=1测试准确率=0.8386训练准确率=0.8309636
# 迭代次数=2测试准确率=0.8661训练准确率=0.8569273
# 迭代次数=3测试准确率=0.8733训练准确率=0.8697818
# 迭代次数=4测试准确率=0.8858训练准确率=0.87929094
# 迭代次数=5测试准确率=0.8919训练准确率=0.8839273
# 迭代次数=6测试准确率=0.8963训练准确率=0.8907818
# 迭代次数=7测试准确率=0.9008训练准确率=0.8932727
# 迭代次数=8测试准确率=0.902训练准确率=0.8970182
# 迭代次数=9测试准确率=0.9087训练准确率=0.9028364